In [1]:
import importlib as il
import numpy as np

from hilde.parsers import read_structure
from hilde.helpers.supercell import make_cubic_supercell
from hilde.templates.lammps import setup_lammps_si, setup_lammps_gan
from hilde.tasks.calculate import calculate_multiple

In [2]:
ph3 = il.import_module('hilde.phono3py.phono3')

In [3]:
atoms = read_structure('../si.in')

In [4]:
sc2, smatrix2 = make_cubic_supercell(atoms, 216)
sc3, smatrix3 = make_cubic_supercell(atoms, 64)
sc3.inform()


Geometry info for:
  input geometry:    Si64_227_64b
  Symmetry prec.:    1e-05
  Number of atoms:   64
  Species:           Si
  Periodicity:       [ True  True  True]
  Lattice:  
    [10.88  0.    0.  ]
    [ 0.   10.88  0.  ]
    [ 0.    0.   10.88]

  Spacegroup:          Fd-3m (227)
  Wyckoff positions:   64*b
  Equivalent atoms:    64*0
  Standard lattice:  
    [5.44 0.   0.  ]
    [0.   5.44 0.  ]
    [0.   0.   5.44]

Cell lengths and angles [Å, °]:
  a, b, c:     10.8800     10.8800     10.8800
  α, β, γ:     90.0000     90.0000     90.0000
  Volume:    1287.9135 Å**3
  Tag  1:                supercell
  Tag  2:                ('smatrix', [-2, 2, 2, 2, -2, 2, 2, 2, -2])


In [5]:
phono3py_settings = {
    'atoms': atoms,
    'fc2_supercell_matrix': smatrix2,
    'fc3_supercell_matrix': smatrix3,
    'cutoff_pair_distance': 2
}

In [6]:
il.reload(ph3)
# help(ph3.preprocess)

<module 'hilde.phono3py.phono3' from '/home/knoop/local/python/lib/python3.7/site-packages/HiLDe-0.1-py3.7-linux-x86_64.egg/hilde/phono3py/phono3.py'>

In [7]:
phonon3, sc2, sc3, scs2, scs3 = ph3.preprocess(**phono3py_settings)

In [8]:
print(f'Number of FC2 displacements: {len(scs2)}')
print(f'Number of FC3 displacements: {len(scs3)}')
print(f'Number of FC3 displacements that were created: {len([sc for sc in scs3 if sc])}')

Number of FC2 displacements: 1
Number of FC3 displacements: 111
Number of FC3 displacements that were created: 3


In [9]:
lammps = setup_lammps_si('.')

## Compute Forces

In [10]:
scs2_computed = calculate_multiple(scs2, lammps, f'{atoms.sysname}/fc2', trajectory_to='fc2.traj')
fc2_forces = ph3.get_forces(scs2_computed)

In [11]:
%time scs3_computed = calculate_multiple(scs3, lammps, f'{atoms.sysname}/fc3', trajectory_to='fc3.traj', trajectory_from='fc3.traj.bak')
fc3_forces = ph3.get_forces(scs3_computed)
len(scs3_computed), len(scs3)

CPU times: user 251 ms, sys: 15.1 ms, total: 266 ms
Wall time: 36.2 ms


(111, 111)

## Produce Force Constants

In [12]:
phonon3.produce_fc2(fc2_forces)

In [13]:
%time phonon3.produce_fc3(fc3_forces)

Computing fc3[ 1, x, x ] using numpy.linalg.pinv with a displacement:
    [ 0.0300  0.0000  0.0000]
Expanding fc3
Cutting-off fc3 (cut-off distance: 2.000000)
Building atom mapping table...
Creating contracted fc3...
CPU times: user 7.06 s, sys: 680 ms, total: 7.74 s
Wall time: 6.88 s


In [14]:
fc2 = phonon3.get_fc2()
fc3 = phonon3.get_fc3()

In [15]:
phonon3_new = ph3.prepare_phono3py(**phono3py_settings,
                                   fc2=fc2,
                                   fc3=fc3,
                                   q_mesh=3*[3])

In [16]:
phonon3_new.run_thermal_conductivity(write_kappa=True)

========================== Warning ==========================
 Phonon frequency of band index 1 at Gamma is calculated to be 0.015460.
 But this frequency is forced to be zero.
========================== Warning ==========================
 Phonon frequency of band index 2 at Gamma is calculated to be 0.015460.
 But this frequency is forced to be zero.
========================== Warning ==========================
 Phonon frequency of band index 3 at Gamma is calculated to be 0.015460.
 But this frequency is forced to be zero.
-------------------- Lattice thermal conducitivity (RTA) --------------------
Lifetime sampling mesh: [ 3 3 3 ]
======================= Grid point 0 (1/4) =======================
q-point: ( 0.00  0.00  0.00)
Number of triplets: 4
Frequency     group velocity (x, y, z)     |gv|
 k*1  ( 0.00  0.00  0.00)
   0.000   (  -0.000   -0.000   -0.000)    0.000
   0.000   (  -0.000    0.000    0.000)    0.000
   0.000   (   0.000    0.000    0.000)    0.000
  16.538   (  -0.0

======================= Grid point 1 (2/4) =======================
q-point: ( 0.33  0.00  0.00)
Number of triplets: 6
Frequency     group velocity (x, y, z)     |gv|
 k*1  ( 0.33  0.00  0.00)
   2.343   (  -8.112    8.112    8.112)   14.050
   2.343   (  -8.112    8.112    8.112)   14.050
   6.990   ( -33.026   33.026   33.026)   57.203
  14.367   (  20.243  -20.243  -20.243)   35.061
  16.180   (   2.439   -2.439   -2.439)    4.224
  16.180   (   2.439   -2.439   -2.439)    4.224
 k*1  ( 0.33  0.00  0.00)
   2.343   (  -8.112    8.112    8.112)   14.050
   2.343   (  -8.112    8.112    8.112)   14.050
   6.990   ( -33.026   33.026   33.026)   57.203
  14.367   (  20.243  -20.243  -20.243)   35.061
  16.180   (   2.439   -2.439   -2.439)    4.224
  16.180   (   2.439   -2.439   -2.439)    4.224
 k*1  ( 0.33  0.00  0.00)
   2.343   (  -8.112    8.112    8.112)   14.050
   2.343   (  -8.112    8.112    8.112)   14.050
   6.990   ( -33.026   33.026   33.026)   57.203
  14.367   (  20.243 

======================= Grid point 4 (3/4) =======================
q-point: ( 0.33  0.33  0.00)
Number of triplets: 6
Frequency     group velocity (x, y, z)     |gv|
 k*1  ( 0.33  0.33  0.00)
   2.418   (  -0.000    0.000   14.384)   14.384
   2.418   (  -0.000    0.000   14.384)   14.384
   8.378   (  -0.000    0.000   61.465)   61.465
  14.385   (   0.000   -0.000  -34.557)   34.557
  15.666   (  -0.000   -0.000   -9.229)    9.229
  15.666   (   0.000   -0.000   -9.229)    9.229
 k*1  ( 0.33  0.33  0.00)
   2.418   (   0.000   -0.000   14.384)   14.384
   2.418   (   0.000   -0.000   14.384)   14.384
   8.378   (   0.000   -0.000   61.465)   61.465
  14.385   (  -0.000    0.000  -34.557)   34.557
  15.666   (   0.000    0.000   -9.229)    9.229
  15.666   (  -0.000    0.000   -9.229)    9.229
 k*1  ( 0.33  0.33  0.00)
   2.418   (  -0.000    0.000   14.384)   14.384
   2.418   (  -0.000    0.000   14.384)   14.384
   8.378   (  -0.000    0.000   61.465)   61.465
  14.385   (   0.000 

======================= Grid point 5 (4/4) =======================
q-point: (-0.33  0.33  0.00)
Number of triplets: 8
Frequency     group velocity (x, y, z)     |gv|
 k*1  (-0.33  0.33  0.00)
   2.954   (   1.135   -1.135    0.000)    1.605
   4.092   (   0.064   -0.064    0.000)    0.091
   9.975   (  26.482  -26.482    0.000)   37.451
  12.438   ( -15.760   15.760    0.000)   22.287
  15.565   (  -2.553    2.553    0.000)    3.610
  15.730   (  -5.243    5.243    0.000)    7.415
 k*1  (-0.33  0.33  0.00)
   2.954   (   1.135   -1.135   -0.000)    1.605
   4.092   (   0.064   -0.064    0.000)    0.091
   9.975   (  26.482  -26.482   -0.000)   37.451
  12.438   ( -15.760   15.760    0.000)   22.287
  15.565   (  -2.553    2.553    0.000)    3.610
  15.730   (  -5.243    5.243    0.000)    7.415
 k*1  (-0.33  0.33  0.00)
   2.954   (   1.135   -1.135    0.000)    1.605
   4.092   (   0.064   -0.064    0.000)    0.091
   9.975   (  26.482  -26.482    0.000)   37.451
  12.438   ( -15.760 

=================== End of collection of collisions ===================
----------- Thermal conductivity (W/m-k) with tetrahedron method -----------
#  T(K)        xx         yy         zz         yz         xz         xy        #ipm
    0.0      0.000      0.000      0.000      0.000      0.000      0.000 3/162
   10.0   6840.802   6840.802   6840.802      0.000      0.000     -0.000 3/162
   20.0   1284.428   1284.428   1284.428      0.000      0.000     -0.000 3/162
   30.0    471.155    471.155    471.155      0.000      0.000     -0.000 3/162
   40.0    237.607    237.607    237.607      0.000      0.000     -0.000 3/162
   50.0    146.928    146.928    146.928      0.000      0.000     -0.000 3/162
   60.0    107.388    107.388    107.388      0.000      0.000     -0.000 3/162
   70.0     89.825     89.825     89.825      0.000      0.000     -0.000 3/162
   80.0     81.712     81.712     81.712      0.000      0.000     -0.000 3/162
   90.0     77.141     77.141     77.141      